Import libraries

In [23]:
import os
import re
import ast
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from itertools import combinations
from sklearn.metrics.pairwise import cosine_similarity

from numba import jit # parallel processing

Define i/o paths

In [10]:
input_data_path = '/kaggle/input/section-sent-texts-with-drug-mentions-ann-200502/'
input_data_file = 'section_sent_texts_with_drug_mentions_ann_200502.csv'

Read and process data

In [11]:
#input_data = pd.read_csv(input_data_path+input_data_file)
input_processed = pd.read_csv(input_data_path+input_data_file)

In [ ]:
#Keep only sentences containing at least 3 words other than those defined below
#This also removes any sentences that do not contain any words at all

# rep = {"text": "", "cite_spans": "", "ref_spans": "", "section": "", "Abstract": "",\
#        "bioRxiv preprint": "", "medRxiv preprint": "", "doi:": ""}
# rep = dict((re.escape(k), v) for k, v in rep.items())
# pattern = re.compile("|".join(rep.keys()))
# sentences_temp = [pattern.sub(lambda m: rep[re.escape(m.group(0))], s) for s in input_data.sentence]
# pattern = re.compile(".*[A-Za-z].*")
# sentences_to_keep = [(bool(re.search(pattern,s))) & (len(s.split(' '))>2) for s in sentences_temp]
# input_processed = input_data.loc[sentences_to_keep,:]
# sentences_to_drop = [not i for i in sentences_to_keep]
# input_excluded = input_data.loc[sentences_to_drop,:]

In [12]:
#Convert w2vVector column from string type to  list
input_processed.w2vVector = [re.sub(',+', ',', ','.join(w.replace('\n','').split(' '))) for w in input_processed.w2vVector]
input_processed.w2vVector = [re.sub('\[,', '', w) for w in input_processed.w2vVector]
input_processed.w2vVector = [re.sub(',\]', '', w) for w in input_processed.w2vVector]
input_processed.w2vVector = [re.sub('\[', '', w) for w in input_processed.w2vVector]
input_processed.w2vVector = [re.sub('\]', '', w) for w in input_processed.w2vVector]
input_processed.w2vVector = input_processed.w2vVector.apply(lambda s: list(ast.literal_eval(s)))

In [14]:
input_processed.to_csv('cord_drug_titles_abstracts_conclusions.csv')
#input_excluded.to_csv('cord_drug_titles_abstracts_conclusions_excluded.csv')

In [15]:
input_processed

,Unnamed: 0,cord_uid,sentence_id,section,sentence,w2vVector,drug_terms_used
0,0,fjmchbew,fjmchbew032256,title,The Author's Response: Case of the Index Patie...,"[0.00944858, -0.05984148, 0.01565553, -0.02196...","lopinavir,ritonavir,lopinavir,ritonavir"
1,1,ba6v30jg,ba6v30jg1633000,Clinical features and outcomes of the patients,The clinical features and laboratory test resu...,"[-0.0305434912, -0.0297232307, 0.147603706, 0....","alanine,creatine"
2,2,ba6v30jg,ba6v30jg1633001,Clinical features and outcomes of the patients,"Fever, cough, and dyspnea were the most common...","[-0.01814028, 0.09488559, 0.08022365, -0.07600...","alanine,creatine"
3,3,ba6v30jg,ba6v30jg1633002,Clinical features and outcomes of the patients,The white blood cell counts varied among these...,"[-0.02238245, -0.02396372, 0.09787606, -0.0587...","alanine,creatine"
4,4,ba6v30jg,ba6v30jg1633003,Clinical features and outcomes of the patients,The alanine aminotransferase and serum creatin...,"[0.0692584217, 0.131579965, 0.0442873463, 0.04...","alanine,creatine"
...,...,...,...,...,...,...,...
13822,13822,647zcjgu,647zcjgu1219583,Discussion,"Among them are captopril, perindopril, ramipri...","[0.11334185, 0.11791535, 0.11523059, 0.0925779...","captopril,lisinopril,amino acids,benazepril,ca..."
13823,13823,647zcjgu,647zcjgu1219584,Discussion,"Although these drugs primarily target ACE, a h...","[0.05551118, -0.01351563, 0.18281737, -0.04538...","captopril,lisinopril,amino acids,benazepril,ca..."
13824,13824,647zcjgu,647zcjgu1219585,Discussion,It should be noted that ACE inhibitors bind to...,"[0.04974999, -0.06304489, 0.10167421, 0.020741...","captopril,lisinopril,amino acids,benazepril,ca..."
13825,13825,647zcjgu,647zcjgu1219586,Discussion,"Nonetheless, these enzymatic inhibitors may in...","[0.00876493, -0.06595846, 0.09507835, -0.01413...","captopril,lisinopril,amino acids,benazepril,ca..."


In [32]:
title_data = input_processed.loc[input_processed.section=='title',:]
abstract_data = input_processed.loc[input_processed.section=='abstract',:]
title_abstract_data = input_processed.loc[(input_processed.section=='title') | (input_processed.section=='abstract'),:]
conclusion_data = input_processed.loc[(input_processed.section!='title') & (input_processed.section!='abstract'),:]

In [18]:
print('Number of papers:', input_processed.cord_uid.nunique())
print('Number of papers with title:', title_data.cord_uid.nunique())
print('Number of papers with abstract:', abstract_data.cord_uid.nunique())
print('Number of papers with conclusion:', conclusion_data.cord_uid.nunique())

Number of papers: 544
Number of papers with title: 108
Number of papers with abstract: 312
Number of papers with conclusion: 301


In [33]:
print('Number of unique sentences under titles:', title_data.sentence.nunique())
print('Number of unique sentence ids under titles:', title_data.sentence_id.nunique())
print('Number of unique sentences under abstracts:', abstract_data.sentence.nunique())
print('Number of unique sentence ids under abstracts:', abstract_data.sentence_id.nunique())

Number of unique sentences under titles: 117
Number of unique sentence ids under titles: 117
Number of unique sentences under abstracts: 3417
Number of unique sentence ids under abstracts: 3590


Calculate cosine similarity between titles

In [46]:
#Average w2v vectors of all sentences falling under a single cord_uid
title_data_final = pd.DataFrame(columns = ['cord_uid','sentence','w2vVector','drugs'])
for cord_uid in title_data.cord_uid.unique():
    title = " ".join(title_data.loc[title_data.cord_uid==cord_uid,'sentence'])
    drugs = ",".join(title_data.loc[title_data.cord_uid==cord_uid,'drug_terms_used'])
    drugs = ",".join(list(set(drugs.split(','))))
    w2vVector = np.mean(list(title_data.loc[title_data.cord_uid==cord_uid,'w2vVector']), axis=0)
    title_data_final = title_data_final.append({'cord_uid':cord_uid,\
                                                'sentence': title,\
                                                'w2vVector': w2vVector,\
                                                'drugs': drugs},\
                                               ignore_index=True)

In [21]:
len(title_data_final)

108

In [ ]:
# pattern = re.compile(".*hydroxychloroquine.*")
# sentences_to_keep = [(bool(re.search(pattern,s.lower()))) for s in title_data_final.sentence]
# drug_title_data = title_data_final.loc[sentences_to_keep,:].reset_index(drop=True)
# len(drug_title_data)

In [ ]:
# title_similarity = pd.DataFrame(columns=['paper1_cord_uid','paper2_cord_uid','title1','title2','similarity_score'])
# jit(nopython=True, parallel=True)
# for i,paper1 in enumerate(drug_title_data.sentence):
#     for j,paper2 in enumerate(drug_title_data.sentence):
#         if i!=j:
#             cos_sim = cosine_similarity(drug_title_data.w2vVector[i].reshape(1,-1),drug_title_data.w2vVector[j].reshape(1,-1))[0][0]
#             title_similarity = title_similarity.append({'paper1_cord_uid':drug_title_data.cord_uid[i],\
#                                                         'paper2_cord_uid':drug_title_data.cord_uid[j],\
#                                                         'title1':paper1,\
#                                                         'title2':paper2,\
#                                                         'similarity_score':cos_sim},\
#                                                ignore_index=True)

In [57]:
title_similarity = pd.DataFrame(columns=['paper1_cord_uid','paper2_cord_uid','title1','title2','drugs1','drugs2','similarity_score'])
jit(nopython=True, parallel=True)
title_pairs = list(combinations(title_data_final.index,2))
for i,j in title_pairs:
    drugs1 = title_data_final.drugs[i].split(',')
    drugs2 = title_data_final.drugs[j].split(',')
    if any(d1 in drugs2 for d1 in drugs1):
        cos_sim = cosine_similarity(title_data_final.w2vVector[i].reshape(1,-1),title_data_final.w2vVector[j].reshape(1,-1))[0][0]
        title_similarity = title_similarity.append({'paper1_cord_uid':title_data_final.cord_uid[i],\
                                                    'paper2_cord_uid':title_data_final.cord_uid[j],\
                                                    'title1':title_data_final.sentence[i],\
                                                    'title2':title_data_final.sentence[j],\
                                                    'similarity_score':cos_sim,\
                                                    'drugs1':title_data_final.drugs[i],\
                                                    'drugs2':title_data_final.drugs[j]},\
                                           ignore_index=True)

title_similarity.to_csv('drug_title_similarity.csv')

Calculate cosine similarity between title+abstracts

In [44]:
#Average w2v vectors of all sentences falling under a single cord_uid
title_abstract_data_final = pd.DataFrame(columns = ['cord_uid','sentence','w2vVector','drugs'])
for cord_uid in title_abstract_data.cord_uid.unique():
    sentences = " ".join(title_abstract_data.loc[title_abstract_data.cord_uid==cord_uid,'sentence'])
    drugs = ",".join(title_abstract_data.loc[title_abstract_data.cord_uid==cord_uid,'drug_terms_used'])
    drugs = ",".join(list(set(drugs.split(','))))
    w2vVector = np.mean(list(title_abstract_data.loc[title_abstract_data.cord_uid==cord_uid,'w2vVector']), axis=0)
    title_abstract_data_final = title_abstract_data_final.append({'cord_uid':cord_uid,\
                                                                'sentence': sentences,\
                                                                'w2vVector': w2vVector,\
                                                                 'drugs': drugs},\
                                                               ignore_index=True)
len(title_abstract_data_final)

352

In [56]:
title_abstract_similarity = pd.DataFrame(columns=['paper1_cord_uid','paper2_cord_uid','text1','text2','drugs1','drugs2','similarity_score'])
jit(nopython=True, parallel=True)
paper_pairs = list(combinations(title_abstract_data_final.index,2))
for i,j in paper_pairs:
    drugs1 = title_abstract_data_final.drugs[i].split(',')
    drugs2 = title_abstract_data_final.drugs[j].split(',')
    if any(d1 in drugs2 for d1 in drugs1):
        cos_sim = cosine_similarity(title_abstract_data_final.w2vVector[i].reshape(1,-1),title_abstract_data_final.w2vVector[j].reshape(1,-1))[0][0]
        title_abstract_similarity = title_abstract_similarity.append({'paper1_cord_uid':title_abstract_data_final.cord_uid[i],\
                                                    'paper2_cord_uid':title_abstract_data_final.cord_uid[j],\
                                                    'text1':title_abstract_data_final.sentence[i],\
                                                    'text2':title_abstract_data_final.sentence[j],\
                                                    'similarity_score':cos_sim,\
                                                    'drugs1':title_abstract_data_final.drugs[i],\
                                                    'drugs2':title_abstract_data_final.drugs[j]},\
                                           ignore_index=True)

title_abstract_similarity.to_csv('drug_title_abstract_similarity.csv')